<a href="https://colab.research.google.com/github/haribharadwaj/notebooks/blob/main/AUDIOLOGY/SpeechIntelligibilityIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computing a Simplified Speech Intelligibility Index (SII)

This notebook demonstrates how to compute a simplified form of the Speech Intelligibility Index (SII) using:

- A given overall speech level (in dB HL).
- A user’s audiogram (in dB HL).
- A standard set of band importance weights and a typical long-term average speech spectrum shape.

**Steps:**

1. **Convert Overall Speech HL to a Band-by-Band Spectrum (dB SPL):**
   - Use a reference LTASS and scale it so that the overall level near 1000 Hz matches the user’s input.
   - Convert from HL to SPL with RETSPLs.

2. **Convert Audiogram to SPL:**
   - Convert user’s threshold in dB HL to dB SPL using RETSPL values.

3. **No Noise (or Add if Needed):**
   - For simplicity, we assume quiet. You can add a noise spectrum similarly if desired.

4. **Calculate Audibility per Band:**
   - Audibility = (Speech_in_Band - Threshold_in_Band) / 30, clipped between 0 and 1.
   - This is a simplified approximation of the audibility function from SII; actual SII computations use more detailed methods.

5. **Apply Band Importance Functions (BIFs):**
   - Multiply each band’s audibility by its importance and sum.

**Output:**
- An SII value between 0 and 1, representing the proportion of speech cues available to the listener.



In [1]:
#@title Audibility Index Function Definitions
import numpy as np

def hl_to_spl(hl_vals, freqs, ret_freqs, ret_values):
    interp_retspl = np.interp(freqs, ret_freqs, ret_values)
    return hl_vals + interp_retspl

def compute_sii(overall_speech_HL, audiogram_freqs, audiogram_levels):
    """
    Compute a simplified SII for Auditec NU-6 recordings using:
    - The importance weights from Studebaker, Sherbecoe, & Gilmore (1993).
    - An ANSI S3.5-based LTASS for normal vocal effort male speech (with slight extrapolation at 125 Hz and 10,000 Hz).

    Parameters
    ----------
    overall_speech_HL : float
        Overall speech level in dB HL at ~1000 Hz.
    audiogram_freqs : array-like
        Frequencies (Hz) at which the audiogram levels are given.
    audiogram_levels : array-like
        Corresponding thresholds in dB HL at those frequencies.

    Returns
    -------
    SII : float
        Estimated Speech Intelligibility Index (0 to 1)

    Notes
    -----
    - This remains a simplified approximation and not a full ANSI S3.5 SII calculation.
    - The LTASS values are taken from ANSI S3.5 (160–8000 Hz) and extrapolated at the edges.
    - The user should ensure that audiogram_freqs are in ascending order and cover or reasonably interpolate within the frequency range of interest.
    """

    audiogram_freqs = np.array(audiogram_freqs, dtype=float)
    audiogram_levels = np.array(audiogram_levels, dtype=float)

    # ER-3A insert earphone RETSPLs (ANSI S3.6 approx)
    ret_freqs = np.array([125,250,500,750,1000,1500,2000,3000,4000,6000,8000])
    ret_values = np.array([45.5,29.5,13.5,9.5,5.5,4.5,2.5,3.0,1.5,4.5,13.0])

    # Bands from the Auditec NU-6 importance function (1/3 octave bands)
    bands = np.array([125,160,200,250,315,400,500,630,800,1000,
                      1250,1600,2000,2500,3150,4000,5000,6300,8000,10000])

    # ANSI-based LTASS (approximate)
    # Using the table above:
    ansi_ltass_dbspl = np.array([
        45.0,  # 125 Hz (extrapolated)
        47.2,  # 160 Hz
        49.1,  # 200 Hz
        51.0,  # 250 Hz
        52.8,  # 315 Hz
        54.6,  # 400 Hz
        56.1,  # 500 Hz
        57.1,  # 630 Hz
        58.1,  # 800 Hz
        59.0,  # 1000 Hz
        59.4,  # 1250 Hz
        59.5,  # 1600 Hz
        59.0,  # 2000 Hz
        57.7,  # 2500 Hz
        56.5,  # 3150 Hz
        55.0,  # 4000 Hz
        53.0,  # 5000 Hz
        50.0,  # 6300 Hz
        47.0,  # 8000 Hz
        44.0   # 10000 Hz (extrapolated)
    ])

    # Convert overall speech HL at 1000 Hz to SPL
    overall_speech_SPL_1000 = hl_to_spl(np.array([overall_speech_HL]),
                                        np.array([1000]), ret_freqs, ret_values)[0]

    # Find band near 1000 Hz
    ref_index = np.argmin(np.abs(bands - 1000))
    diff = overall_speech_SPL_1000 - ansi_ltass_dbspl[ref_index]
    speech_spl_bands = ansi_ltass_dbspl + diff

    # Interpolate audiogram to these band frequencies
    threshold_hl_bands = np.interp(bands, audiogram_freqs, audiogram_levels,
                                   left=audiogram_levels[0], right=audiogram_levels[-1])
    threshold_spl_bands = hl_to_spl(threshold_hl_bands, bands, ret_freqs, ret_values)

    # Assume no noise
    noise_spl_bands = np.full(len(bands), -999.0)

    # Compute audibility
    audibility = np.zeros(len(bands))
    for i in range(len(bands)):
        effective_floor = max(threshold_spl_bands[i], noise_spl_bands[i])
        aud = (speech_spl_bands[i] - effective_floor)/30.0
        audibility[i] = np.clip(aud, 0, 1)

    # Importance function from Studebaker et al. (1993) for Auditec NU-6
    bif_percent = np.array([0.73,0.95,1.30,2.11,3.44,5.17,7.37,6.58,6.44,6.64,
                            8.02,9.87,11.71,9.32,7.83,5.62,3.37,1.77,1.04,0.72])
    bif = bif_percent / np.sum(bif_percent)

    # Compute SII
    SII = np.sum(audibility * bif)
    return SII

In [2]:
#@title Transfer Function to Estimate Percent Correct Scores

def nu6_transfer_from_sii(A, P=1.00, Q=0.404, N=3.334):
    """
    Compute the predicted NU-6 score (proportion correct) from the Audibility Index (A),
    using the transfer function parameters from Studebaker, Sherbecoe, & Gilmore (1993).

    Parameters
    ----------
    A : float or array-like
        Audibility Index (or SII-like measure) between 0 and 1.
    P : float
        Proficiency factor (default 1.00).
    Q : float
        Fitting constant (default 0.404).
    N : float
        Fitting constant controlling steepness (default 3.334).

    Returns
    -------
    S : float or ndarray
        Predicted NU-6 score in percent correct (0 to 100).
    """
    A = np.array(A, dtype=float)
    S = 100. * (1 - 10**(-(A * P / Q)))**N
    return S



## Example Usage

In [3]:
# Example usage:

audiogram_freqs = [250, 500, 1000, 2000, 3000, 4000, 6000, 8000]
audiogram_levels = [25, 30, 40, 50, 50, 55, 60, 80]
overall_speech_HL = 70
sii_value = compute_sii(overall_speech_HL, audiogram_freqs, audiogram_levels)
predicted_score = nu6_transfer_from_sii(sii_value)
print(f"Predicted Word Score given Audibility Index of {sii_value:.2f} is: {predicted_score:.1f}%")

Predicted Word Score given Audibility Index of 0.74 is: 95.3%
